# Grammar & Translation scheme

entrada —> sent {sent.s->evaluar()} entrada  <br>
        —> épsilon  <br>

sent —> PRINT exp ‘;’    {sent.s := new NodoPrint(expr.s);} <br>
     —> ID ASIGN exp ‘;’ {sent.s := new NodoAsign(ID.lexval, expr.s);} <br>

exp  —> exp OR term {exp.s := new NodoOR(exp1.s, term.s)}<br>
     —> term {exp.s = term.s} <br>

term —> term AND fact {term.s := new NodoAND(term1.s, fact.s)}<br>
     —> fact {term.s = fact.s}<br>

fact —> NOT fact {fact.s := new NodoNOT(fatc1.s)}<br>
     —> ‘(‘ exp ‘)’ {fact.s := exp.s}<br>
     —> ID {fact.s := new NodoID(Id.lexval)}<br>
     —> TRUE {fact.s := new NodoTrue()}<br>
     —> FALSE {fact.s := new NodoFalse()}<br>

# Grammar adaptation

entrada —> sent {sent.s->evaluar()} entrada  <br>
        —> épsilon  <br>

sent —> PRINT exp ‘;’    {sent.s := new NodoPrint(expr.s);} <br>
     —> ID ASIGN exp ‘;’ {sent.s := new NodoAsign(ID.lexval, expr.s);} <br>

expr -> term {expr'.h := term.s;} expr' {expr.s := expr'.s;} <br>

expr' -> OR term {expr'1.h := new NodoOR(expr'.h, term.s);} expr'1 {expr'.s := expr'1.s;} <br>
      -> epsilon    {expr'.s := expr'.h;} <br>

term -> fact {term'.h := fact.s;} term' {term.s := term'.s;} <br>

term' -> AND fact {term'1.h := new NodoConj(term'.h, fact.s);} term'1 {term'.s := term'1.s;} <br>
      -> epsilon    {term'.s := term'.h;} <br>

fact —> NOT fact {fact.s := new NodoNOT(fatc1.s)}<br>
     —> ‘(‘ exp ‘)’ {fact.s := exp.s}<br>
     —> ID {fact.s := new NodoID(Id.lexval)}<br>
     —> TRUE {fact.s := new NodoTrue()}<br>
     —> FALSE {fact.s := new NodoFalse()}<br>

# First and Follow sets

First(fact)    = {NOT, '(', ID, VERDADERO, FALSO}<br>
First(term')   = {AND, epsilon}<br>
First(term)    = First(fact) = {NOT, '(', ID, VERDADERO, FALSO}<br>
First(expr')   = {OR, epsilon}<br>
First(exp)     = First(term) = First(fact) = {NOT, '(', ID, VERDADERO, FALSO}<br>
First(sent)    = {PRINT, ID}<br>
First(entrada) = First(sent) U {epsilon} = {PRINT, ID, epsilon}<br>

Follow(entrada) = {EOF}<br>
Follow(sent)    = First(Entrada) U Follow(Entrada) = {PRINT, ID, EOF}<br>
Follow(expr)    = {';', ')'}<br>
Follow(expr')   = Follow(exp) = Follow(exp') = {';', ')'}<br>
Follow(term)    = First(expr') U Follow(expr) U Follow(expr') = {OR, ';', ')'}<br>
Follow(term')   = Follow(term) U Follow(term') = {OR, ';'}<br>
Follow(fact)    = First(term') U Follow(term) U Follow(term') = {AND, OR, ';', ')'}<br>

# Bottom - up implementation

## Lexer

In [1]:
from sly import Lexer

ta = None
ind = 0
toks = None
tokenlist = None

class CalcLexer(Lexer):
    tokens = {ID, NOT, OR, AND, TRUE, FALSE, PRINT, ASIG}
    ignore = ' \t'
    literals = { ';', '(', ')'}

    # Regular expression rules for tokens
    ASIG          = r':='
    ID            = r'[a-zA-Z_][a-zA-Z0-9_]*'
    ID['or']      = OR
    ID['and']     = AND
    ID['not']     = NOT
    ID['true']    = TRUE
    ID['false']   = FALSE
    ID['print']   = PRINT
    
    @_(r'\n+')
    def newline(self, t):
        self.lineno += t.value.count('\n')

    def error(self, t):
        print("Illegal character '%s'" % t.value[0])
        self.index += 1

## Classes

In [ ]:
tabla = {}

class Nodo:
    def evaluar():
        pass
    def toStr():
        pass
#fin de la clase abstracta Nodo

class NodoTrue(Nodo):
    def evaluar(self): 
        return True
    def toStr(): 
        return "true"
#fin de la clase NodoTrue

class NodoFalse(Nodo):
    def evaluar(self): 
        return False
    def toStr(): 
        return "false"
#fin de la clase NodoFalse

class NodoId(Nodo):
    def __init__(self, n):
        self.nombre = n
    def evaluar(self): 
        global tabla
        return  tabla[self.nombre]
    def toStr(): 
        return "ID(" + self.nombre + ")"
#fin de la clase NodoId

class NodoNOT(Nodo):
    def __init__(self, c):
        self.comp = c
    def evaluar(self):
        return  not self.comp.evaluar()
    def toStr(): 
        return "NO(" + self.comp.toStr() + ")"
#fin de la clase NodoNOT

class NodoAND(Nodo):
    def __init__(self, i, d):
        self.izq = i
        self.der = d
    def evaluar(self):
        return  self.izq.evaluar() and self.der.evaluar()
    def toStr(): 
        return "and(" + self.izq.toStr() + "," + self.der.toStr() + ")"
#fin de la clase NodoAND

class NodoOR(Nodo):
    def __init__(self, i, d):
        self.izq = i
        self.der = d
    def evaluar(self):
        return  self.izq.evaluar() or self.der.evaluar()
    def toStr(): 
        return "or(" + self.izq.toStr() + "," + self.der.toStr() + ")"
#fin de la clase NodoOR

class NodoAsign(Nodo): 
    def __init__(self, n, e):
        self.nombre = n
        self.expr = e
    def evaluar(self):
        global tabla
        tabla[self.nombre] = self.expr.evaluar()
        return tabla[self.nombre]
    def toStr():
        return "tabla[" + self.nombre + "]=" + self.expr.toStr()
#fin de la clase NodoAsign

class NodoPrint(Nodo):
    def __init__(self, e):
        self.expr = e
    def evaluar(self):
        temporal = self.expr.evaluar()
        print("resultado = ", int(temporal))
        return temporal
    def toStr(): 
        return "print(" + self.expr.toStr() + ")"
#fin de la clase NodoPrint


## Parser

In [ ]:
def yyerror(msj):
    print("Error sistactico", msj)
#fin de yyerror() 

def cuadra(obj): 
    global ta, ind, tokenlist
    if ta.type == obj:
        #print("cuadro ta = ", ta.value) 
        ind += 1
        if ind < len(tokenlist):
            ta = tokenlist[ind]
            #print("==> nuevo ta = ", ta.value)
    else:
        yyerror("en cuadra"); 
#fin de cuadra() 

def fact():
    global ta, toks
    if ta.type == toks[4]: #NOT
        cuadra("NOT")
        fact1_s = fact()
        return NodoNOT(fact1_s)
    else:
        if ta.type == toks[3]: #ID
            IDlexval = ta.value
            cuadra("ID")
            return NodoId(IDlexval)
        else: 
            if ta.type == toks[7]: #true
                cuadra("TRUE")
                return NodoTrue()
            else:
                if ta.type == toks[2]: #false
                    cuadra("FALSE")
                    return NodoFalse()
                else:
                    if ta.value == '(':
                        cuadra('(')
                        expr_s = expr()
                        cuadra(')')
                        return expr_s
                    else:
                        error("en fact");
#fin de fact()

def term_prima(term_prima_h):
    global ta, toks
    if ta.type == toks[0]: #AND
        cuadra("AND")
        fact_s = fact()
        term_prima_1_h = NodoAND(term_prima_h, fact_s)
        return term_prima(term_prima_1_h)
    else:
        if ta.value == ';' or ta.value == ')' or ta.type == toks[5]:
            return term_prima_h;
        else:
            yyerror("en term_prima");
#fin de term_prima()

def term():
    global ta, toks
    if ta.type == toks[4] or ta.type == toks[7] or ta.type == toks[2] or ta.type == toks[3] or ta.value == '(':
        facts_s = fact()
        term_prima_h = facts_s
        term_prima_s = term_prima(term_prima_h)
        return term_prima_s
    else:
        yyerror("en term");
#fin de term()

def expr_prima(expr_prima_h):
    global ta, toks
    if ta.type == toks[5]: #OR
        cuadra("OR")
        term_s = term()
        expr_prima_1_h = NodoOR(expr_prima_h, term_s)
        return expr_prima(expr_prima_1_h)
    else:
        if ta.value == ';' or ta.value == ')':
            return expr_prima_h
        else:
            yyerror("en expr_prima")
#fin de expr_prima()

def expr():
    global ta, toks
    if ta.type == toks[4] or ta.type == toks[7] or ta.type == toks[2] or ta.type == toks[3] or ta.value == '(':
        term_s = term()
        expr_prima_h = term_s
        expr_prima_s = expr_prima(expr_prima_h)
        return expr_prima_s
    else:
        yyerror("en expr")
#fin de expr()


def sent():
    global ta, toks
    if ta.type == toks[6]: #PRINT
        cuadra("PRINT")
        expr_s = expr()
        cuadra(';')
        return NodoPrint(expr_s)
    else:
        if ta.type == toks[3]: #ID
            IDlexval = ta.value
            cuadra("ID")
            cuadra("ASIG")
            expr_s = expr()
            cuadra(';')
            return NodoAsign(IDlexval, expr_s)
        else:
            yyerror("en sent")
# fin de sent()

def entrada(): 
    global ta, toks, tokenlist
    if ta.type == toks[6] or ta.type == toks[3]:
        sent_s = sent()
        sent_s.evaluar()
        entrada()
    else: 
        if ind < len(tokenlist):  #fin de la entrada
            yyerror("En entrada")
#fin de entrada()


if __name__ == '__main__':
    global ta, toks, tokenlist, ind
    lexer = CalcLexer()
    toks = CalcLexer.tokens
    toks = sorted(toks)
    while True:
        try:
            data = input('data type list > ')
        except EOFError:
            break
        if data:
            tokenlist = list(lexer.tokenize(data))
            ta = tokenlist[ind]
            entrada()
        ind = 0